In [3]:
import psycopg2
from datetime import date
import datetime
import pandas as pd


db_credentials = {
    'dbname': 'Web_Extention',
    'user': 'postgres',
    'password': '2080',
    'host': 'localhost',
    'port': '5432'
}

connection = psycopg2.connect(**db_credentials)

# Create a cursor to interact with the database
cursor = connection.cursor()

# SELECT query to retrieve data from the 'bbc_embeddings' table
today = datetime.date.today()

select_query_todays_articles = f"SELECT * FROM bbc_daily_links;"

# Execute the query
cursor.execute(select_query_todays_articles)

# Fetch all the rows from the result set
rows = cursor.fetchall()

# Create a DataFrame from the fetched data
columns = [desc[0] for desc in cursor.description]  # Get column names
data_frame_from_table = pd.DataFrame(rows, columns=columns)
#data_frame_from_table["Title_some_content"] = data_frame_from_table.apply(lambda row: row["title"] + row["article_content"][100:500], axis=1)

In [15]:
from sitemaps_utils import *

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [20]:
data_frame_from_table["cleaned_content"] = [clean_text(article) for article in data_frame_from_table.article_content]

In [26]:
from sentence_transformers import SentenceTransformer

model_name = 'distilbert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(model_name)
embeddings = model.encode(data_frame_from_table["cleaned_content"])

In [27]:
data_frame_from_table["all_content_embedding"] = list(embeddings)

In [9]:
embeddings = model.encode(data_frame_from_table["article_content"])
data_frame_from_table["Embedding_all_conetent"] = list(embeddings)
data_frame_from_table['Embedding'] = data_frame_from_table['Embedding'].apply(lambda x: [float(val) for val in x])

In [48]:
data_frame_from_table.cleaned_content[1][100:500]

' ago15 minut agofrom section rugbi leaguecomments0ethan ryan score tri hull kr stun challeng cup semifin win wigan warrior last week got scoresheet castleford tiger fridaybetfr super leaguehul kr 16 34tri opac kennedi linnett minchella batchelor ryan goal miln 5castleford 10 16tri tate broadbent eden goal dean 2hull kr score score six tri comfort beat castleford tigerstri tom opac rhi kennedi kane'

In [42]:
connection = psycopg2.connect(**db_credentials)

# Create a cursor to interact with the database
cursor = connection.cursor()
for idx, Value in enumerate(data_frame_from_table.all_content_embedding):
    Value = [float(i) for i in Value]
    insert_query = "UPDATE bbc_embeddings SET cleaned_content_embedding = %s WHERE id = %s;"
    cursor.execute(insert_query, (list(Value),str(idx + 1)))

connection.commit()    
connection.close()

In [81]:
cleaned = [clean_text(data_frame_from_table.article_content[1].split(".")[i]) for i in range(len(data_frame_from_table.article_content[1].split(".")))]